<a href="https://colab.research.google.com/github/bryanttsai96/Diamond-Price/blob/main/Diamond_(drop_variables).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px

In [ ]:
df = pd.read_excel("UV6248-XLS-ENG.xls", sheet_name = "Raw Data")
df = df.rename(columns={'Carat Weight': 'CaratWeight'})
df = df.dropna()
df1= df.copy()

Check the number of observations across categories in all variables.

For problematic categories, combine the ''rare categories" together.

Try this on your own with your teammates. If you do not how, ask ChatGPT.

In [ ]:
df1.Clarity.unique()
print(df1.Clarity.unique())
FL_count2 = df1['Clarity'].value_counts()
print(FL_count2)

# FL is quite low and consider combine or drop it
df1['Clarity']= df1['Clarity'].replace({'IF': 'IF_FL','FL': 'IF_FL'})
df1.Clarity.unique()
print(df1.Clarity.unique())

Step 2.2 Visual Exploration of the Data
Explore impact of different categorical variables on our data.

In [ ]:
fig = px.scatter(df1,
                 x="CaratWeight",
                 y="Price",
                 color = 'Color',
                 height = 350
                )
fig.show()


In [ ]:
#explore whether taking a log of X or Y helps the relationship.
from math import log
#by doing copy the original data won't be change by all the number being log
df_log = df.copy()
df_log['Price']=df_log['Price'].transform(log)
#df_log['CaratWeight']=df_log['CaratWeight'].transform(log)
print(df_log['Price'].head(10))

fig = px.scatter(df_log,
                 x="CaratWeight",
                 y="Price",
                 color = 'Color',
                 height = 350,
                 labels={'Price':'Log of Price'#, 'CaratWeight': 'Log of CaratWeight'
                         }
                )
fig.show()

Create dummy variables for categorical variables using pd.get_dummies():

In [ ]:
df1 = pd.get_dummies(data=df, columns = ['Color', 'Report'], drop_first=True)
df1.head() # Check how the data looks now
# ['Color','Clarity','Cut','Polish','Symmetry','Report']

Look at the correlation matrix to see if we have potential multicollinearity

Encode ordinal categorical variables using an increasing scale

In [ ]:
scale_map_cut = {'Fair':1, 'Good':2, 'Very Good':3, 'Ideal':4,  'Signature-Ideal':5}
scale_map_clarity = {'SI1':1, 'VVS2':2, 'VVS1':3, 'VS2':4, 'VS1':5, 'IF_FL':6}
scale_map_polish = {'G':1, 'VG':2, 'EX':3, 'ID':3}

df1["Cut"] = df1["Cut"].replace(scale_map_cut)
df1["Clarity"] = df1["Clarity"].replace(scale_map_clarity)
df1["Polish"] = df1["Polish"].replace(scale_map_polish)
df1["Symmetry"] = df1["Symmetry"].replace(scale_map_polish)
df1.tail()

In [ ]:
print(df1.dtypes)
df1.corr().style.background_gradient(cmap='RdBu_r', axis=None)
df1.drop(['Price','ID'], axis=1).corr().style.background_gradient(cmap='coolwarm', axis=None)
#"axis=None" option above indicates that the colors are assigned based on the values in the whole matrix
# Other good color maps: 'RdBu_r' & 'PuOr_r' & 'coolwarm'

Compute Variance Inflation Factors (VIF)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe

vif_data1 = pd.DataFrame()
vif_data1["feature"] = df1.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data1["VIF"] = [variance_inflation_factor(df1.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df1.drop(['Price','ID'], axis=1).columns))]
print(vif_data1)

Try mean-centering variables
Try creating combinations of variables
Try dropping variables

In [ ]:
#drop 'Symmetry','Polish', 'Cut'
vif_data2 = pd.DataFrame()
vif_data2["feature"] = df1.drop(['Price','ID','Symmetry','Polish'], axis=1).columns


# calculating VIF for each feature
vif_data2["VIF"] = [variance_inflation_factor(df1.drop(['Price','ID','Symmetry','Polish'], axis=1).values, i)
                          for i in range(len(df1.drop(['Price','ID', 'Symmetry','Polish'], axis=1).columns))]
print(vif_data2)

Split the data into X and Y.
The vector of Y ("dependent") variable should contain the Price. The matrix of X ("independent") variables should contain everything we will use to predict Y

In [ ]:
Y = df1[(['Price'])]
Y.head() # it's always a good idea to peak at your output

In [ ]:
X = df1.drop(['Price','ID', 'Report_GIA','Symmetry','Polish'], axis=1)
#X = df[['CaratWeight']]
X.dtypes

Build regression models
3.1 Try a linear model

In [ ]:
# In this package, by default, the regression will have no intercept, hence we need to manually add it to the X matrix, and call the result X_const
X_const = sm.add_constant(X)
X_const.head()
# Fit a linear regression model with vector Y as dependent and matrix X_sm as independent
lm = sm.OLS(Y, X_const).fit()

# Display the summary of model results
print(lm.summary())

Plot the residuals

In [ ]:
# Compute the residuals
results = pd.DataFrame()
results['Price'] = df['Price']
results['prediction_lm'] = lm.fittedvalues
results['residual_lm'] = lm.resid

fig = px.scatter(
    results, x='prediction_lm', y='residual_lm', height = 350,
    labels={'prediction_lm':'Predicted values using the Linear Model', 'residual_lm':'Residuals using the Linear Model'}
)
fig.show()

log-linear model to the case data

In [ ]:
from math import exp, log
Y_log = Y['Price'].transform(log)
log_linear_model = sm.OLS(Y_log, X_const).fit()
print(log_linear_model.summary())

In [ ]:
# Compute residuals
results['prediction_llm'] = log_linear_model.fittedvalues
results['residual_llm'] = log_linear_model.resid

fig = px.scatter(
    results, x='prediction_llm', y='residual_llm', height = 350,
    labels={'prediction_llm':'Predicted values using the Log-Linear Model', 'residual_llm':'Residuals'}
)
fig.show()

Fit a log-log model to the case data

In [ ]:
from math import exp, log
Y_log = Y['Price'].transform(log)
#duplicate x for making the original data clean
X_log= X.copy()
X_log['CaratWeight'] = X_log['CaratWeight'].transform(log)
X_log_const= sm.add_constant(X_log)
X_log_const.head()
log_log_model = sm.OLS(Y_log, X_log_const).fit()
print(log_log_model.summary())

In [ ]:
# Compute residuals
results['prediction_llm'] = log_log_model.fittedvalues
results['residual_llm'] = log_log_model.resid

fig = px.scatter(
    results, x='prediction_llm', y='residual_llm', height = 350,
    labels={'prediction_llm':'Predicted values using the Log-Log Model', 'residual_llm':'Residuals'}
)
fig.show()

Cross-validation
The main machine learning principle that allows to answer this question -- cross-validation: splitting the data into training (80%) and testing (20%) subsets, training on the former and testing on the latter. Find the MAPE

In [ ]:
# Redefine X and Y for the training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Add a constant to the X's:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

X_train_const.head()

In [ ]:
# Fit a linear regression model to the training data
lm = sm.OLS(Y_train, X_train_const).fit()

# Use the trained model to predict the prices for the testing data. Call the vector of predicted prices Y_pred
Y_pred = lm.predict(X_test_const)
percent_errors = np.abs((Y_test['Price'] - Y_pred) / Y_test['Price']) *100
print("Linear Model MAPE = ", np.mean(percent_errors), "%")

In [ ]:
# Fit a log-linear regression model to the training data       #nearly 2 time better than linear model
Y_train_log = Y_train['Price'].transform(log)
llm = sm.OLS(Y_train_log, X_train_const).fit()

Y_pred_llm = np.exp(llm.predict(X_test_const))
percent_errors = np.abs((Y_test['Price'] - Y_pred_llm) / Y_test['Price']) *100
print("Log-Linear Model MAPE = ", np.mean(percent_errors), "%")

In [ ]:
# Fit a log-log regression model to the training data
X_train_log = X_train.copy()
X_train_log['CaratWeight'] = X_train_log['CaratWeight'].transform(log)
X_train_log_const = sm.add_constant(X_train_log)

X_test_log = X_test.copy()
X_test_log['CaratWeight'] = X_test_log['CaratWeight'].transform(log)
X_test_log_const = sm.add_constant(X_test_log)


loglog = sm.OLS(Y_train_log, X_train_log_const).fit()

Y_pred_loglog = np.exp(loglog.predict(X_test_log_const))
percent_errors = np.abs((Y_test['Price'] - Y_pred_loglog) / Y_test['Price']) *100
print("Log-Log Model MAPE = ", np.mean(percent_errors), "%")